Keras是一个基于Python编写的高层神经网络API，凭借用户友好性、模块化以及易扩展等有点大受好评，考虑到Keras的优良特性以及它的受欢迎程度，TensorFlow2.0中将Keras的代码吸收了进来，化身为tf.keras模块供用户使用。

使用tf.keras提供的高层API，可以轻松得完成建模三部曲——模型构建、训练、评估等工作。下面我们分别来说说如何使用tf.keras完成这三部曲。

# 1 模型构建

我们知道，神经网络模型就是层的堆叠，tf.keras提供的Sequential类对象就是层容器，可以轻松实现对层的堆叠，创建网络模型。用Sequential创建一个全连接网络模型：

In [3]:
import tensorflow as tf
from tensorflow import keras  # 为方便使用，keras一般单独导入
from tensorflow.keras import layers

In [2]:
model = tf.keras.Sequential()
# 往模型中添加一个有64个神经元组成的层，激活函数为relu:
model.add(layers.Dense(64, activation='relu'))
# 再添加一个:
model.add(layers.Dense(64, activation='relu'))
# 添加一个有10个神经元的softmax层作为输出层:
model.add(layers.Dense(10, activation='softmax'))

也可以在使用Sequential实例化模型时，通过传入由层组成的列表来添加层。我们换一种方式实现上面的模型构建过程,两种方式是完全等效的：

In [6]:
model = tf.keras.Sequential([
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')]
)

你看，用tf.keras创建一个模型，就是这么简单，只需要往Sequential中传入一个个tf.keras.layers定义的层就好了。进一步的，我们研究一下tf.keras.layers怎么个性化地创建层。

定义神经网络层通过tf.keras.layers模块中的Dense类实现，Dense类构造参数如下：
- units：指定神经元个数，必须是一个正整数。
- activation：激活函数，可以是可以是一个可调用对象或标识一个对象的字符串
- use_bias：布尔型，是否使用是否使用偏置项
- kernel_initializer和bias_initializer：权值、偏置初始化方法,可以是一个可调用对象或标识一个对象的字符串
- kernel_regularizer和bias_regularizer：对权值、偏置进行正则化的方法，可以是一个可调用对象或标识一个对象的字符串
- activity_regularizer：对层的输出进行正则化的方法，可以是一个可调用对象或标识一个对象的字符串
- kernel_constraint和bias_constraint：对权值矩阵、偏置矩阵的约束方法，可以是一个可调用对象或标识一个对象的字符串

In [6]:
# 有64个神经元，激活函数为sigmoid的层
layers.Dense(64, activation='sigmoid')
# 或者:
layers.Dense(64, activation=tf.keras.activations.sigmoid)

# 对权值矩阵进行正则化:
layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l1(0.01))

# 对偏置向量进行正则化:
layers.Dense(64, bias_regularizer=tf.keras.regularizers.l2(0.01))

# 指定权值随机正交初始化:
layers.Dense(64, kernel_initializer='orthogonal')

# 指定偏置为常数:
layers.Dense(64, bias_initializer=tf.keras.initializers.Constant(2.0))

# 2 训练模型

## 2.1 配置：compile()

建立好模型之后，接下来当然是要进行训练模型了。不过，在训练前还需要做一些配置工作，例如指定优化器、损失函数、评估指标等，这些配置参数的过程一般通过tf.keras.Model.compile方法进行，先来熟悉一下tf.keras.Model.compile方法的三个常用参数：
- optimizer：tf.keras.optimizers模块中的优化器**实例化对象**，例如 tf.keras.optimizers.Adam或 tf.keras.optimizers.SGD的实例化对象，当然也可以使用字符串来指代优化器，例如'adam'和'sgd'。
- loss：损失函数，例如交叉熵、均方差等，通常是tf.keras.losses模块中定义的可调用对象，也可以用用于指代损失函数的字符串。
- metrics：元素为评估方法的list，通常是定义在tf.keras.metrics模块中定义的可调用对象，也可以用于指代评估方法的字符串。

在知道怎么配置模型训练参数后，就可以根据实际应用情况合理选择优化器、损失函数、评估方法等：

In [7]:
# 回归模型
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),  # 指定优化器，学习率为0.01
              loss='mse',       # 指定均方差作为损失函数
              metrics=['mae'])  # 添加绝对值误差作为评估方法

# 分类模型
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(),  # 分类模型多用交叉熵作为损失函数
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

通过compile()配置好模型后，就可以开始训练了。tf.keras中提供了fit()方法对模型进行训练，先来看看fit()方法的主要参数：
- x和y：训练数据和目标数据
- epochs：训练周期数，每一个周期都是对训练数据集的一次完整迭代
- batch_size：簇的大小，一般在数据集是numpy数组类型时使用
- validation_data：验证数据集，模型训练时，如果你想通过一个额外的验证数据集来监测模型的性能变换，就可以通过这个参数传入验证数据集
- verbose：日志显示方式，verbose=0为不在标准输出流输出日志信息,verbose=1为输出进度条记录，verbose=2为每个epoch输出一行记录
- callbacks：回调方法组成的列表，一般是定义在tf.keras.callbacks中的方法
- validation_split：从训练数据集抽取部分数据作为验证数据集的比例，是一个0到1之间的浮点数。这一参数在输入数据为dataset对象、生成器、keras.utils.Sequence对象是无效。
- shuffle：是否在每一个周期开始前打乱数据

下面分别说说如何使用fit()方法结合numpy数据和tf.data.Dataset数据进行模型训练。

In [8]:
import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

model.fit(data, labels, epochs=10, batch_size=32)

Train on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 1s 554us/sample - loss: 206.2688 - categorical_accuracy: 0.1050
Epoch 2/10
1000/1000 [==============================] - 0s 34us/sample - loss: 911.8347 - categorical_accuracy: 0.0990
Epoch 3/10
1000/1000 [==============================] - 0s 30us/sample - loss: 1879.7505 - categorical_accuracy: 0.0980
Epoch 4/10
1000/1000 [==============================] - 0s 28us/sample - loss: 3141.3959 - categorical_accuracy: 0.0940
Epoch 5/10
1000/1000 [==============================] - 0s 36us/sample - loss: 4673.7791 - categorical_accuracy: 0.1010
Epoch 6/10
1000/1000 [==============================] - 0s 36us/sample - loss: 6526.8757 - categorical_accuracy: 0.0960
Epoch 7/10
1000/1000 [==============================] - 0s 31us/sample - loss: 8571.8533 - categorical_accuracy: 0.1020
Epoch 8/10
1000/1000 [==============================] - 0s 33us/sample - loss: 11070.1039 - categorical_accuracy: 0.0970
Epoch 9/10
1000/10

如何使用验证数据集的话，可以这样：

In [10]:
import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

val_data = np.random.random((100, 32))
val_labels = np.random.random((100, 10))

model.fit(data, labels, epochs=10, batch_size=32,
          validation_data=(val_data, val_labels))  # 验证数据集以元组的形式传入

Train on 1000 samples, validate on 100 samples
Epoch 1/10
1000/1000 [==============================] - 0s 34us/sample - loss: 67219.8359 - categorical_accuracy: 0.0960 - val_loss: 55306.6777 - val_categorical_accuracy: 0.1000
Epoch 2/10
1000/1000 [==============================] - 0s 32us/sample - loss: 73732.5724 - categorical_accuracy: 0.0920 - val_loss: 89920.2088 - val_categorical_accuracy: 0.1100
Epoch 3/10
1000/1000 [==============================] - 0s 50us/sample - loss: 79956.1480 - categorical_accuracy: 0.1020 - val_loss: 101092.6750 - val_categorical_accuracy: 0.1000
Epoch 4/10
1000/1000 [==============================] - 0s 36us/sample - loss: 84322.9844 - categorical_accuracy: 0.0970 - val_loss: 117610.5700 - val_categorical_accuracy: 0.1000
Epoch 5/10
1000/1000 [==============================] - 0s 38us/sample - loss: 91992.0751 - categorical_accuracy: 0.1130 - val_loss: 94200.0838 - val_categorical_accuracy: 0.1000
Epoch 6/10
1000/1000 [==============================] - 

# 3 评估与预测

是骡子是马，拉出来溜溜就知道了，训练好的模型性能如何，评估测试一下就知道了。可以使用模型自带的evaluate()方法和predict()方法对模型进行评估和预测。

In [12]:
# 如果是numpy数据，可以这么使用
data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

model.evaluate(data, labels, batch_size=32)

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[153591.27975, 0.093]

In [13]:
# 如果数Dataset对象，可以这么使用
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)

model.evaluate(dataset)

32/32 [==============================] - 0s 579us/step - loss: 153946.2378 - categorical_accuracy: 0.0930


[153946.23779296875, 0.093]

使用predict()方法进行预测：

In [14]:
# numpy数据
result = model.predict(data, batch_size=32)
print(result.shape)

(1000, 10)


In [16]:
# dataset数据
result = model.predict(dataset)
print(result.shape)

(1000, 10)
